In [116]:
import requests
import pytz
import json
from datetime import datetime, time as datetime_time, timedelta
import calendar
import pandas as pd


In [117]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r


In [118]:
def make_api_request(stop_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/OperationMonitoring/4.0/PassingTimeByPoint/" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [119]:
def parse_waiting_times(json_payload, destination):
    passing_times = json_payload['points'][0]['passingTimes']
    str_expected_arrival_times = [passing_time['expectedArrivalTime'] for passing_time in passing_times if passing_time['destination']['fr']==destination]
    print(str_expected_arrival_times)
    expected_arrival_times = [datetime.fromisoformat(str_expected_arrival_time) for str_expected_arrival_time in str_expected_arrival_times]
    return expected_arrival_times 

In [120]:
def compute_time_diff(start, end):
    if isinstance(start, datetime_time): # convert to datetime
        assert isinstance(end, datetime_time)
        start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
    if start <= end: # e.g., 10:33:26-11:15:49
        return end - start
    else: # end < start e.g., 23:55:00-00:25:00
        end += timedelta(1) # +day
        assert end > start
        return end - start

In [121]:
from datetime import datetime, time as datetime_time, timedelta
import pytz


class TimeUtils:

    @staticmethod
    def get_current_localized_time():
        current_time_utc = datetime.utcnow()
        return pytz.utc.localize(current_time_utc)

    @staticmethod
    def compute_time_diff(start, end):
        if isinstance(start, datetime_time): # convert to datetime
            assert isinstance(end, datetime_time)
            start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
        if start <= end: # e.g., 10:33:26-11:15:49
            return TimeUtils._convert_timedelta(end - start)
        else: # end < start e.g., 23:55:00-00:25:00
            end += timedelta(1) # +day
            assert end > start
            return TimeUtils._convert_timedelta(end - start)

    @staticmethod
    def _convert_timedelta(duration):
        duration_dict = {}
        days, seconds = duration.days, duration.seconds
        duration_dict['days'] = days
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = (seconds % 60)
        duration_dict['hours'] = hours
        duration_dict['minutes'] = minutes
        duration_dict['seconds'] = seconds

        return duration_dict


In [122]:
from dataclasses import dataclass, field
from typing import List, Optional, Any

from dataclasses_json import dataclass_json, config, LetterCase

from datetime import datetime

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Message:
    fr: str = ""
    nl: str = ""
        
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Destination:
    fr: str = ""
    nl: str = ""

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PassingTime:
    destination: Optional[Destination] = None
    message: Optional[Message] = None
    line_id: str = ""
    expected_arrival_time: str = ""
    arriving_in_dict: dict = field(init=False, repr=False)
        
    def __post_init__(self):
        self.expected_arrival_time = datetime.fromisoformat(self.expected_arrival_time)
        current_localized_time = TimeUtils.get_current_localized_time()
        self.arriving_in_dict = TimeUtils.compute_time_diff(current_localized_time, self.expected_arrival_time)


    def __str__(self):
        return f'{self.arriving_in_dict}'

        

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PointPassingTimes:
    passing_times: List[PassingTime] = field(default_factory=list)
    point_id: str = ""



In [123]:
legrand_stop_id = "1047F"
bearer = '543567766625859b7088d85b1cdbadad'
response = make_api_request(legrand_stop_id, bearer)
response

{'points': [{'passingTimes': [{'destination': {'fr': 'ROODEBEEK',
      'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-04T15:32:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-04T15:12:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}},
    {'destination': {'fr': 'ROODEBEEK', 'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-04T15:56:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-04T15:43:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}}],
   'pointId': '1047'}]}

In [124]:
passages = PointPassingTimes.schema().load(response['points'], many=True) 

passages

[PointPassingTimes(passing_times=[PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 4, 15, 32, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 4, 15, 12, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 4, 15, 56, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 4, 15, 43, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))))], point

In [125]:
expected_arrival_times = parse_waiting_times(response, 'ROODEBEEK' )

current_time_utc = datetime.utcnow()
current_time_be = pytz.utc.localize(current_time_utc)
tram_arrival_time = expected_arrival_times[0]

waiting_time = compute_time_diff(current_time_be, tram_arrival_time)
print(waiting_time)


['2020-04-04T15:32:00+02:00', '2020-04-04T15:56:00+02:00']
0:26:37.877885


In [126]:
def get_shapefiles():
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/Files/2.0/Shapefiles" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [127]:
from enum import Enum

class RouteType(Enum):
    TRAMWAY = 0
    METRO = 1
    BUS = 3

In [128]:
ROUTE_DATAPATH = "gtfs/routes.txt"
STOP_DATAPATH = "gtfs/stops.txt"
routes_df = pd.read_csv(ROUTE_DATAPATH)
stops_df = pd.read_csv(STOP_DATAPATH)

In [141]:
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LinePoint:
    id: str = ""
    order: int = 0
    stop_name: str = ""

    def __post_init__(self):
        try:
            self.stop_name = stops_df[stops_df["stop_id"] == self.id].iloc[0]["stop_name"]
        except:
            self.stop_name = ""
            


@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LineDetails:
    destination: Optional[Destination] = None
    direction: str = ""
    line_id: str = ""
    points: List[LinePoint] = field(default_factory=list)
    route_type: Optional[RouteType] = None

    def __post_init__(self):
        self.route_type = RouteType(
            routes_df[routes_df["route_short_name"] == self.line_id].iloc[0][
                "route_type"
            ]
        )


In [142]:
def make_line_info_api_request(line_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/NetworkDescription/1.0/PointByLine/" + line_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()


In [143]:
line_info_api_response = make_line_info_api_request("93", bearer)
line_info_api_response

{'lines': [{'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
   'direction': 'Suburb',
   'lineId': '93',
   'points': [{'id': '5400', 'order': 1},
    {'id': '5402', 'order': 2},
    {'id': '5399F', 'order': 3},
    {'id': '5403', 'order': 4},
    {'id': '5010', 'order': 5},
    {'id': '6501', 'order': 6},
    {'id': '6505', 'order': 7},
    {'id': '6504', 'order': 8},
    {'id': '4130F', 'order': 9},
    {'id': '6100', 'order': 10},
    {'id': '6122', 'order': 11},
    {'id': '1073', 'order': 12},
    {'id': '6103F', 'order': 13},
    {'id': '6178', 'order': 14},
    {'id': '5762', 'order': 15},
    {'id': '1646F', 'order': 16},
    {'id': '6012G', 'order': 17},
    {'id': '6412F', 'order': 18},
    {'id': '6014', 'order': 19},
    {'id': '6437F', 'order': 20},
    {'id': '6308F', 'order': 21},
    {'id': '6309F', 'order': 22},
    {'id': '6310F', 'order': 23},
    {'id': '6311', 'order': 24},
    {'id': '6312', 'order': 25},
    {'id': '6313', 'order': 26},
    {'id': '6314', 'ord

In [147]:
lines = LineDetails.schema().load(line_info_api_response['lines'], many=True) 

lines[0].route_type.name

'TRAMWAY'

In [152]:
line_details_serialized = [line.to_dict() for line in lines]
line_details_serialized

[{'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
  'direction': 'Suburb',
  'lineId': '93',
  'points': [{'id': '5400', 'order': 1, 'stopName': ''},
   {'id': '5402', 'order': 2, 'stopName': 'STIENON'},
   {'id': '5399F', 'order': 3, 'stopName': 'POLYCL. BRUGMANN'},
   {'id': '5403', 'order': 4, 'stopName': 'HOPITAL BRUGMANN'},
   {'id': '5010', 'order': 5, 'stopName': 'GUILL. DE GREEF'},
   {'id': '6501', 'order': 6, 'stopName': ''},
   {'id': '6505', 'order': 7, 'stopName': ''},
   {'id': '6504', 'order': 8, 'stopName': ''},
   {'id': '4130F', 'order': 9, 'stopName': 'BOCKSTAEL'},
   {'id': '6100', 'order': 10, 'stopName': 'MOORSLEDE'},
   {'id': '6122', 'order': 11, 'stopName': 'PRINC. CLEMENTINE'},
   {'id': '1073', 'order': 12, 'stopName': 'OUTRE-PONTS'},
   {'id': '6103F', 'order': 13, 'stopName': 'JULES DE TROOZ'},
   {'id': '6178', 'order': 14, 'stopName': 'MASUI'},
   {'id': '5762', 'order': 15, 'stopName': 'THOMAS'},
   {'id': '1646F', 'order': 16, 'stopName': 'LIEDTS'},


In [145]:
stop_name = stops_df[stops_df["stop_id"] == "5402"]
stop_name

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1843,5402,NaN,STIENON,NaN,50.890647,4.330038,NaN,NaN,0,NaN


In [154]:
lines

[LineDetails(destination=Destination(fr='LEGRAND', nl='LEGRAND'), direction='Suburb', line_id='93', points=[LinePoint(id='5400', order=1, stop_name=''), LinePoint(id='5402', order=2, stop_name='STIENON'), LinePoint(id='5399F', order=3, stop_name='POLYCL. BRUGMANN'), LinePoint(id='5403', order=4, stop_name='HOPITAL BRUGMANN'), LinePoint(id='5010', order=5, stop_name='GUILL. DE GREEF'), LinePoint(id='6501', order=6, stop_name=''), LinePoint(id='6505', order=7, stop_name=''), LinePoint(id='6504', order=8, stop_name=''), LinePoint(id='4130F', order=9, stop_name='BOCKSTAEL'), LinePoint(id='6100', order=10, stop_name='MOORSLEDE'), LinePoint(id='6122', order=11, stop_name='PRINC. CLEMENTINE'), LinePoint(id='1073', order=12, stop_name='OUTRE-PONTS'), LinePoint(id='6103F', order=13, stop_name='JULES DE TROOZ'), LinePoint(id='6178', order=14, stop_name='MASUI'), LinePoint(id='5762', order=15, stop_name='THOMAS'), LinePoint(id='1646F', order=16, stop_name='LIEDTS'), LinePoint(id='6012G', order=17